## BERT для вопросно-ответных систем

Стандартный датасет для тренировки QA моделей (или question-answering моделей) называется **SQUAD**[2] и расшифровывается как Stanford Question Answering Dataset. 

Он включает в себя вопросы и ответы на них по достаточно разнообразной тематике. 
- Вопросы формировались на основе статей или отрывков из статьи из Википедии. 
- Ответ на каждый вопрос представляет собой сегмент текста или промежуток из соответствующего отрывка. 
- В новой версии датасета (в 2.0 версии) даже возможны вопросы без ответа. 
  - Имеется в виду, что есть вопросы, для ответа на которые недостаточно информации в предложенном фрагменте текста. 
 
Таким образом, алгоритм составления этого датасета был следующим. 
- Асессор читал фрагмент текста из Википедии (как правило, всего несколько абзацев), 
- формулировал вопрос по прочитанному и фиксировал правильный ответ. 

На большинство вопросов присутствует несколько вариантов ответа. Но, как правило, это всего лишь переформулировки одного и того же, по смыслу, варианта ответа. 

Например, на вопрос "когда началась эпоха Возрождения в Италии" можно ответить "14 век", "в 14 веке", "в начале 14 века" или как-то ещё, использовав слова "14" и "век". 

Учитывая специфику нашего датасета, на вход нейросети мы будем подавать не только сам вопрос, но и соответствующий фрагмент текста, параграф текста. А в качестве выхода нейросети будем ожидать две позиции в тексте: начало ответа и конец ответа. 


[1] https://stepik.org/lesson/268748/step/7?unit=249768  
[2] https://rajpurkar.github.io/SQuAD-explorer/

In [1]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

Скачайте датасет (SQuAD) [отсюда](https://rajpurkar.github.io/SQuAD-explorer/). Для выполенения семинара Вам понадобятся файлы `train-v2.0.json` и `dev-v2.0.json`.

Склонируйте репозиторий https://github.com/huggingface/transformers (воспользуйтесь скриптом `clone_pytorch_transformers.sh`) и положите путь до папки `examples` в переменную `PATH_TO_EXAMPLES`. 

In [2]:
import os

PATH_TO_TRANSFORMERS_REPO = '../transformers/'
os.environ['PATH_TO_TRANSFORMER_REPO'] = PATH_TO_TRANSFORMERS_REPO

In [3]:
# ! wget https://raw.githubusercontent.com/Samsung-IT-Academy/stepik-dl-nlp/master/clone_pytorch_transformers.sh
# ! bash clone_pytorch_transformers.sh $PATH_TO_TRANSFORMERS_REPO

Мы клоновали древнюю версию `transformers` и хотим использовать ее вместо уже установленного пакета. 
При запуске питон окружения пути для поиска модулей собираются в `sys.path`  в следующем порядке (поиск модулей также будет идти в этом порядке):
1. Домашний каталог программы.
2. Каталоги PYTHONPATH (если установлены).
   - в уже запущенном окружении переустанавливать для данного окружения смысла уже нет (в `sys.path` не попадет)
3. Каталоги стандартной библиотеки.
4. Содержимое любых файлов .pth (при их наличии).
5. Подкаталог site-packages, где размещаются сторонние расширения.

In [4]:
import sys

sys.path, os.environ.get('PYTHONPATH', None)

(['/mnt/INT_STORAGE/PYTHON/projects/stepik.org/DataScience/Нейронные сети и обработка текста/stepik-dl-nlp',
  '/usr/local/lib/python39.zip',
  '/usr/local/lib/python3.9',
  '/usr/local/lib/python3.9/lib-dynload',
  '',
  '/home/user1/envs/py39/lib/python3.9/site-packages',
  '/home/user1/envs/py39/lib/python3.9/site-packages/torchtext-0.11.0a0-py3.9-linux-x86_64.egg',
  '/home/user1/envs/py39/lib/python3.9/site-packages/torchaudio-0.10.1+6f539cf-py3.9-linux-x86_64.egg',
  '/home/user1/.local/lib/python3.9/site-packages'],
 None)

Аппендить в `sys.path` при уже установленном пакете тоже нет смысла, т.к. каталоги пакетов будут раньше, и питон найдет этот модуль там.

In [5]:
import sys

PATH_TO_EXAMPLES = os.path.join(PATH_TO_TRANSFORMERS_REPO, 'examples')

sys.path.insert(1, PATH_TO_EXAMPLES)
sys.path.insert(1, PATH_TO_TRANSFORMERS_REPO)

# Импорт модулей

In [6]:
import torch
import tqdm
import json

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)

from run_squad import train, load_and_cache_examples

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

from transformers import (WEIGHTS_NAME, BertConfig, XLNetConfig, XLMConfig,
                          BertForQuestionAnswering, BertTokenizer)

from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Готовые токенайзер и модель

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [8]:
# если Вы не хотите запускать файн-тюнинг, пропустите блок "Дообучение",
# подгрузите веса уже дообученной модели и переходите к блоку "Оценка качества"

# скачайте веса с Google-диска и положите их в папку models
# https://drive.google.com/drive/folders/1-DR30q7MF-gZ51TDx596dAOhgh-uOAPj?usp=sharing

In [9]:
if torch.cuda.is_available():
    model.cuda()
    model.load_state_dict(torch.load('models/bert_squad_1epoch.pt')) # если у вас есть GPU
else:
    model.load_state_dict(torch.load('models/bert_squad_1epoch.pt', map_location=device)) # если GPU нет

### Дообучение

- ~7 часов на 5 эпох

В класс "train_opts" выписаны ВСЕ параметры, на которые стоит обратить внимание перед запуском модели.


In [10]:
# !pip install dataclasses
from dataclasses import dataclass

@dataclass
class TRAIN_OPTS:
    train_file : str = './datasets/SQUAD/train-v2.0.json'    # SQuAD json-файл для обучения
    predict_file : str = './datasets/SQUAD/dev-v2.0.json'    # SQuAD json-файл для тестирования
    model_type : str = 'bert'               # тип модели (может быть  'bert', 'xlnet', 'xlm', 'distilbert')
    model_name_or_path : str = 'bert-base-uncased' # путь до предобученной модели или название модели из ALL_MODELS
    output_dir : str = '/tmp' # путь до директории, где будут храниться чекпоинты и предсказания модели
    device : str = 'cuda' # cuda или cpu
    n_gpu : int = 1 # количество gpu для обучения
    cache_dir : str = '' # где хранить предобученные модели, загруженные с s3
        
    # Если true, то в датасет будут включены вопросы, на которые нет ответов.
    version_2_with_negative : bool = True
    # Если (null_score - best_non_null) больше, чем порог, предсказывать null.
    null_score_diff_threshold : float = 0.0
    # Максимальная длина входной последовательности после WordPiece токенизации. Sequences 
    # Последовательности длиннее будут укорочены, для более коротких последовательностей будет использован паддинг
    max_seq_length : int = 384
    # Сколько stride использовать при делении длинного документа на чанки
    doc_stride : int = 128
    # Максимальное количество токенов в вопросе. Более длинные вопросы будут укорочены до этой длины
    max_query_length : int = 128 #
        
    do_train : bool = True
    do_eval : bool = True
        
    # Запускать ли evaluation на каждом logging_step
    evaluate_during_training : bool = True
    # Должно быть True, если Вы используете uncased модели
    do_lower_case : bool = True #
    
    per_gpu_train_batch_size : int = 8 # размер батча для обучения
    per_gpu_eval_batch_size : int = 8 # размер батча для eval
    learning_rate : float = 5e-5 # learning rate
    gradient_accumulation_steps : int = 1 # количество шагов, которые нужно сделать перед backward/update pass
    weight_decay : float = 0.0 # weight decay
    adam_epsilon : float = 1e-8 # эпсилон для Adam
    max_grad_norm : float = 1.0 # максимальная норма градиента
    num_train_epochs : float = 5.0 # количество эпох на обучение
    max_steps : int = -1 # общее количество шагов на обучение (override num_train_epochs)
    warmup_steps : int = 0 # warmup 
    n_best_size : int = 5 # количество ответов, которые надо сгенерировать для записи в nbest_predictions.json
    max_answer_length : int = 30 # максимально возможная длина ответа
    verbose_logging : bool = True # печатать или нет warnings, относящиеся к обработке данных
    logging_steps : int = 5000 # логировать каждые X шагов
    save_steps : int = 5000 # сохранять чекпоинт каждые X шагов
        
    # Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number
    eval_all_checkpoints : bool = True
    no_cuda : bool = False # не использовать CUDA
    overwrite_output_dir : bool = True # переписывать ли содержимое директории с выходными файлами
    overwrite_cache : bool = True # переписывать ли закешированные данные для обучения и evaluation
    seed : int = 42 # random seed
    local_rank : int = -1 # local rank для распределенного обучения на GPU
    fp16 : bool = False # использовать ли 16-bit (mixed) precision (через NVIDIA apex) вместо 32-bit"
    # Apex AMP optimization level: ['O0', 'O1', 'O2', and 'O3'].
    # Подробнее тут: https://nvidia.github.io/apex/amp.html
    fp16_opt_level : str = '01'

Все варианты моделей, с которыми мы можем работать с помощью библиотеки pytorch-transformers

In [11]:
ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) \
                  for conf in (BertConfig, XLNetConfig, XLMConfig)), ())
ALL_MODELS

('bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'xlnet-base-cased',
 'xlnet-large-cased',
 'xlm-mlm-en-2048',
 'xlm-mlm-ende-1024',
 'xlm-mlm-enfr-1024',
 'xlm-mlm-enro-1024',
 'xlm-mlm-tlm-xnli15-1024',
 'xlm-mlm-xnli15-1024',
 'xlm-clm-enfr-1024',
 'xlm-clm-ende-1024',
 'xlm-mlm-17-1280',
 'xlm-mlm-100-1280')

In [12]:
# args = TRAIN_OPTS()
# train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)

In [13]:
# train(args, train_dataset, model, tokenizer)

Сохраняем веса дообученной модели на диск, чтобы в следующий раз не обучать модель заново.

In [14]:
# torch.save(model.state_dict(), 'models/bert_squad_final_5epoch.pt')

Подгрузить веса модели можно так:

In [15]:
# model.load_state_dict(torch.load('models/bert_squad_5epochs.pt'))

# Оценка качества работы модели

Выделим подмножество вопросов, чтобы быстрее считалось

In [16]:
PATH_TO_DEV_SQUAD = './datasets/SQUAD/dev-v2.0.json'
PATH_TO_SMALL_DEV_SQUAD = './datasets/SQUAD/small_dev-v2.0.json'

with open(PATH_TO_DEV_SQUAD, 'r') as iofile:
    full_sample = json.load(iofile)
    
small_sample = {
    'version': full_sample['version'],
    'data': full_sample['data'][:1]
}

with open(PATH_TO_SMALL_DEV_SQUAD, 'w') as iofile:
    json.dump(small_sample, iofile)

Параметры, которые нам будут нужны для evaluation

In [17]:
max_seq_length = 384
outside_pos = max_seq_length + 10
doc_stride = 128
max_query_length = 64
max_answer_length = 30

- Грузим малую выборку
- Конвертируем в признаки (используем тот же токенайзер, что при обучении)
- Вытаскиваем из признаков нужные переменные (похожи на те, что в классификации предложений)
  - input_ids номера токенов
  - input_mask 1/0 = токен/паддинг
  - cls_index индекс классификационного токена
  - p_mask 1 - токены, которых не может быть в ответе, 0 - могут быть в ответе

In [18]:
examples = read_squad_examples(
    input_file=PATH_TO_SMALL_DEV_SQUAD,
    is_training=False,
    version_2_with_negative=True)

features = convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=False,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    cls_token_at_end=False
)

input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

example_index = torch.arange(input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(input_ids, input_mask, segment_ids, example_index, cls_index, p_mask)

100%|██████████| 208/208 [00:01<00:00, 135.44it/s]


Создаем их них объект `DataLoader`

In [19]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

Просчет модели
- распаковываем батч на переменные и передаём полученные данные в нашу модель, чтобы получить предсказание модели
- ответ модели собираем в `all_results`

In [27]:
from tqdm.notebook import tqdm  # !pip install ipywidgets; jupyter nbextension enable --py widgetsnbextension

def to_list(tensor):
    return tensor.detach().cpu().tolist()

all_results = []
for idx, batch in enumerate(tqdm(eval_dataloader, desc="Evaluating")):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }
        inputs['token_type_ids'] = batch[2]
        example_indices = batch[3]
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        result = RawResult(unique_id    = unique_id,
                           start_logits = to_list(outputs[0][i]),
                           end_logits   = to_list(outputs[1][i]))
        all_results.append(result)

Evaluating:   0%|          | 0/27 [00:00<?, ?it/s]

In [43]:
all_results[0].unique_id, all_results[0].start_logits[:5], all_results[0].end_logits[:5]

(1000000000,
 [0.5573755502700806,
  -5.768959045410156,
  -4.9304986000061035,
  -6.4673590660095215,
  -8.117743492126465],
 [0.7398626208305359,
  -8.749387741088867,
  -5.381954669952393,
  -6.655617713928223,
  -8.192229270935059])

Сформируем читабельный ответ из аппроксимации модели

In [31]:
n_best_size = 5
do_lower_case = True
output_prediction_file = './results/output_1best_file'
output_nbest_file = './results/output_nbest_file'
output_na_prob_file = './results/output_na_prob_file'
verbose_logging = True
version_2_with_negative = True
null_score_diff_threshold = 0.0

In [44]:
# Генерируем файл с n лучшими ответами `output_nbest_file`
write_predictions(examples, features, all_results, n_best_size,
                    max_answer_length, do_lower_case, output_prediction_file,
                    output_nbest_file, output_na_prob_file, verbose_logging,
                    version_2_with_negative, null_score_diff_threshold)
pass

`EVAL_OPTS` — по структуре оно очень похоже на класс `TRAIN_OPTS`

Считаем метрики используя официальный SQuAD script[1]

[1] https://rajpurkar.github.io/SQuAD-explorer/

In [33]:
# Считаем метрики используя официальный SQuAD script
evaluate_options = EVAL_OPTS(data_file=PATH_TO_SMALL_DEV_SQUAD,
                             pred_file=output_prediction_file,
                             na_prob_file=output_na_prob_file)
results = evaluate_on_squad(evaluate_options)

{
  "exact": 68.26923076923077,
  "f1": 71.22710622710623,
  "total": 208,
  "HasAns_exact": 76.04166666666667,
  "HasAns_f1": 82.45039682539682,
  "HasAns_total": 96,
  "NoAns_exact": 61.607142857142854,
  "NoAns_f1": 61.607142857142854,
  "NoAns_total": 112,
  "best_exact": 72.59615384615384,
  "best_exact_thresh": -5.180886268615723,
  "best_f1": 74.75274725274724,
  "best_f1_thresh": -2.038421154022217
}


Посмотрим глазами на вопросы и предсказанные БЕРТом ответы:

In [45]:
with open('./results/output_nbest_file', 'r') as iofile:
    predicted_answers = json.load(iofile)

In [46]:
questions = {}
for paragraph in small_sample['data'][0]['paragraphs']:
    for question in paragraph['qas']:
        questions[question['id']] = {
            'question': question['question'],
            'answers': question['answers'],
            'paragraph': paragraph['context']
        }

In [47]:
for q_num, (key, data) in enumerate(predicted_answers.items()):
    gt = '' if len(questions[key]['answers']) == 0 else questions[key]['answers'][0]['text']
    print('Вопрос {0}:'.format(q_num+1), questions[key]['question'])
    print('-----------------------------------')
    print('Ground truth:', gt)
    print('-----------------------------------')   
    print('Ответы, предсказанные БЕРТом:')
    preds = ['{0}) '.format(ans_num + 1) + answer['text'] + \
             ' (уверенность {0:.2f}%)'.format(answer['probability']*100) \
             for ans_num, answer in enumerate(data)]
    print('\n'.join(preds))
#     print('-----------------------------------')   
#     print('Параграф:', questions[key]['paragraph'])
    print('\n\n')

Вопрос 1: In what country is Normandy located?
-----------------------------------
Ground truth: France
-----------------------------------
Ответы, предсказанные БЕРТом:
1) France (уверенность 99.22%)
2) a region in France (уверенность 0.37%)
3) France. (уверенность 0.15%)
4) region in France (уверенность 0.14%)
5) France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway (уверенность 0.12%)
6)  (уверенность 0.00%)



Вопрос 2: When were the Normans in Normandy?
-----------------------------------
Ground truth: 10th and 11th centuries
-----------------------------------
Ответы, предсказанные БЕРТом:
1) 10th and 11th centuries (уверенность 44.30%)
2) in the 10th and 11th centuries (уверенность 29.12%)
3) the 10th and 11th centuries (уверенность 25.49%)
4) 11th centuries (уверенность 0.83%)
5) 10th and 11th (уверенность 0.25%)
6)  (уверенность 0.00%)



Вопрос 3: From which countries did the Norse originate?
-------------

# Что еще

1. Можно сравнить метрики после 5 эпох и после 1 эпохи, сильного отличия нет, а по времени это 7 часов и 1,5 часа

2. Код выше, это разбор готового скрипта из библиотеки `transformers`

        export SQUAD_DIR=/path/to/SQUAD

        python run_squad.py \
        --model_type bert \
        --model_name_or_path bert-base-cased \
        --do_train \
        --do_eval \
        --do_lower_case \
        --train_file $SQUAD_DIR/train-v1.1.json \
        --predict_file $SQUAD_DIR/dev-v1.1.json \
        --per_gpu_train_batch_size 12 \
        --learning_rate 3e-5 \
        --num_train_epochs 2.0 \
        --max_seq_length 384 \
        --doc_stride 128 \
        --output_dir /tmp/debug_squad/

3. Все это может послужить отправной точкой в более подробном освоении кода библиотеки и запуске BERT, XL трансформера, GPT-2 или любых других моделей для решения ваших задач.